# Import

In [1]:
// %install-swiftpm-flags -c release
// %install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.5")' SwiftCoreMLTools
// %install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

In [2]:
import Foundation
import TensorFlow
// import SwiftCoreMLTools
// import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
//     try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
// }

# Data Ingestion

In [4]:
let data = try String(contentsOfFile:"./data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

# Data Transformation

## Split Numerical Categorical Features

In [5]:
let categoricalColumns = [3, 8]
let numericalColumns = [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]
let numCategoricalFeatures = categoricalColumns.count
let numNumericalFeatures = numericalColumns.count
let numLabels = 1

assert(numColumns == numCategoricalFeatures + numNumericalFeatures + 1)

// Get Categorical Features
let allCategoriesValues = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
                                .reduce(into: Array(repeating: [Int32](), count: 2)){ total, value in
                                    total[0].append(value[0])
                                    total[1].append(value[1]) }
                                .map{ Set($0).sorted() }

let embeddingSizes = allCategoriesValues.map{ $0.count }

let categoricalFeatures = dataFeatures.map{ row in categoricalColumns.map{ Int32(row[$0]) } }
let oneHotCategoricalFeatures:[[[Int32]]] = categoricalFeatures.map{ catArray in
    var oneHotArray = [[Int32]]()
    
    for i in 0..<catArray.count {
        var oneHot = Array(repeating: Int32(0), count: allCategoriesValues[i].count)
        if let pos = allCategoriesValues[i].firstIndex(where: { $0 == catArray[i] }){
            oneHot[pos] = 1
        }
        oneHotArray.append(oneHot)
    }
    
    return oneHotArray
}

// Get Numerical Features
let numericalFeatures = dataFeatures.map{ row in numericalColumns.map{ row[$0] } }

## Split Train and Test

In [6]:
let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

func matrixTranspose<T>(_ matrix: [[T]]) -> [[T]] {
    if matrix.isEmpty {return matrix}
    var result = [[T]]()
    for index in 0..<matrix.first!.count {
        result.append(matrix.map{$0[index]})
    }
    return result
}

let xCategoricalAllTrain = matrixTranspose(Array(oneHotCategoricalFeatures[0..<numTrainRecords])).map{ Array($0.joined()) }
let xCategoricalAllTest = matrixTranspose(Array(oneHotCategoricalFeatures[numTrainRecords...])).map{ Array($0.joined()) }
let xNumericalAllTrain = Array(Array(numericalFeatures[0..<numTrainRecords]).joined())
let xNumericalAllTest = Array(Array(numericalFeatures[numTrainRecords...]).joined())
let yAllTrain = Array(Array(dataLabels[0..<numTrainRecords]).joined())
let yAllTest = Array(Array(dataLabels[numTrainRecords...]).joined())

let XCategoricalTrain = xCategoricalAllTrain.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTrainRecords, embeddingSizes[offset]]))
}
let XCategoricalTest = xCategoricalAllTest.enumerated().map{ (offset, element) in 
    Tensor<Int32>(element).reshaped(to: TensorShape([numTestRecords, embeddingSizes[offset]]))
}
let XNumericalTrainDeNorm = Tensor<Float>(xNumericalAllTrain).reshaped(to: TensorShape([numTrainRecords, numNumericalFeatures]))
let XNumericalTestDeNorm = Tensor<Float>(xNumericalAllTest).reshaped(to: TensorShape([numTestRecords, numNumericalFeatures]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, numLabels]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, numLabels]))

## Normalize Numerical Features

In [7]:
let mean = XNumericalTrainDeNorm.mean(alongAxes: 0)
let std = XNumericalTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[ 2.0137098,  14.197531,   9.523555, 0.53213036,  6.3311296,   64.47929,  4.1678762,  353.68396,
    18.03163,  379.84735,  11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.11449408,  0.7311985,  29.000755,  2.1797554,  132.14561,
    2.217345,  40.494495,   6.852825]]


In [8]:
let XNumericalTrain = (XNumericalTrainDeNorm - mean)/std
let XNumericalTest = (XNumericalTestDeNorm - mean)/std

In [9]:
print("Training shapes \(XNumericalTrain.shape) \(XCategoricalTrain[0].shape) \(XCategoricalTrain[1].shape) \(YTrain.shape)")
print("Testing shapes  \(XNumericalTest.shape) \(XCategoricalTest[0].shape) \(XCategoricalTest[1].shape) \(YTest.shape)")

Training shapes [405, 11] [405, 2] [405, 9] [405, 1]
Testing shapes  [101, 11] [101, 2] [101, 9] [101, 1]


# Model

In [10]:
struct MultiInputs<N: Differentiable, C>: Differentiable {
  var numerical: N
  
  @noDerivative
  var categorical: C

  @differentiable
  init(numerical: N, categorical: C) {
    self.numerical = numerical
    self.categorical = categorical
  }
}

struct RegressionModel: Module {
    var numericalLayer = Dense<Float>(inputSize: 11, outputSize: 32, activation: relu)
    var embedding1 = Embedding<Float>(vocabularySize: 2, embeddingSize: 2)
    var embedding2 = Embedding<Float>(vocabularySize: 9, embeddingSize: 5)
    var embeddingLayer = Dense<Float>(inputSize: (4 + 45), outputSize: 64, activation: relu)
    var allInputConcatLayer = Dense<Float>(inputSize: (32 + 64), outputSize: 128, activation: relu)
    var hiddenLayer = Dense<Float>(inputSize: 128, outputSize: 32, activation: relu)
    var outputLayer = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: MultiInputs<[Tensor<Float>], [Tensor<Int32>]>) -> Tensor<Float> {
        let numericalInput = numericalLayer(input.numerical[0])
        let embeddingOutput1 = embedding1(input.categorical[0])
        let embeddingOutput1Reshaped = embeddingOutput1.reshaped(to: 
            TensorShape([embeddingOutput1.shape[0], embeddingOutput1.shape[1] * embeddingOutput1.shape[2]]))
        let embeddingOutput2 = embedding2(input.categorical[1])
        let embeddingOutput2Reshaped = embeddingOutput2.reshaped(to: 
            TensorShape([embeddingOutput2.shape[0], embeddingOutput2.shape[1] * embeddingOutput2.shape[2]]))
        let embeddingConcat = Tensor<Float>(concatenating: [embeddingOutput1Reshaped, embeddingOutput2Reshaped], alongAxis: 1)
        let embeddingInput = embeddingLayer(embeddingConcat)
        let allConcat = Tensor<Float>(concatenating: [numericalInput, embeddingInput], alongAxis: 1)
        return allConcat.sequenced(through: allInputConcatLayer, hiddenLayer, outputLayer)
    }
}

var model = RegressionModel()

# Training

In [11]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [12]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [13]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                         categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                       XCategoricalTrain[1][batchStart..<batchEnd]])
            let logits = model(multiInput)
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let multiInput = MultiInputs(numerical: [XNumericalTrain[batchStart..<batchEnd]],
                                     categorical: [XCategoricalTrain[0][batchStart..<batchEnd],
                                                   XCategoricalTrain[1][batchStart..<batchEnd]])
        let logits = model(multiInput)
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 561.0152, MAE: 20.997587
Epoch 2: MSE: 357.30536, MAE: 15.783238
Epoch 3: MSE: 196.12015, MAE: 10.911716
Epoch 4: MSE: 118.944565, MAE: 7.5713677
Epoch 5: MSE: 91.62054, MAE: 6.2539334
Epoch 6: MSE: 77.22692, MAE: 5.648963
Epoch 7: MSE: 67.58952, MAE: 5.2488885
Epoch 8: MSE: 59.54583, MAE: 4.878821
Epoch 9: MSE: 52.289066, MAE: 4.5536394
Epoch 10: MSE: 47.661724, MAE: 4.332935
Epoch 11: MSE: 43.345066, MAE: 3.9419541
Epoch 12: MSE: 40.67019, MAE: 3.8101451
Epoch 13: MSE: 35.991726, MAE: 3.748586
Epoch 14: MSE: 35.48568, MAE: 3.4018588
Epoch 15: MSE: 33.962975, MAE: 3.3601305
Epoch 16: MSE: 31.686687, MAE: 3.411875
Epoch 17: MSE: 30.413136, MAE: 3.1732402
Epoch 18: MSE: 28.818247, MAE: 3.0740914
Epoch 19: MSE: 28.525036, MAE: 3.0059052
Epoch 20: MSE: 26.9489, MAE: 3.0503619
Epoch 21: MSE: 26.777122, MAE: 2.985069
Epoch 22: MSE: 24.033762, MAE: 2.895381
Epoch 23: MSE: 25.595734, MAE: 2.844287
Epoch 24: MSE: 23.6858, MAE: 2.954912
Epoch 25: MSE: 23.96878, MAE: 2.9035416
Epoc

Epoch 202: MSE: 5.8218985, MAE: 1.4854475
Epoch 203: MSE: 4.2392707, MAE: 1.3973844
Epoch 204: MSE: 6.0961485, MAE: 1.5515734
Epoch 205: MSE: 5.300884, MAE: 1.4435271
Epoch 206: MSE: 4.7900896, MAE: 1.4255292
Epoch 207: MSE: 4.626521, MAE: 1.4776471
Epoch 208: MSE: 5.7123137, MAE: 1.4614472
Epoch 209: MSE: 5.2897954, MAE: 1.4243444
Epoch 210: MSE: 5.2005415, MAE: 1.4611669
Epoch 211: MSE: 4.5226965, MAE: 1.3935704
Epoch 212: MSE: 6.136378, MAE: 1.5076531
Epoch 213: MSE: 4.2014503, MAE: 1.3140907
Epoch 214: MSE: 5.2831087, MAE: 1.5371026
Epoch 215: MSE: 4.7702336, MAE: 1.4558277
Epoch 216: MSE: 4.3522005, MAE: 1.3818283
Epoch 217: MSE: 4.4555917, MAE: 1.4910088
Epoch 218: MSE: 4.4346466, MAE: 1.4388064
Epoch 219: MSE: 5.9545927, MAE: 1.526848
Epoch 220: MSE: 4.3214087, MAE: 1.3611051
Epoch 221: MSE: 5.382268, MAE: 1.5569413
Epoch 222: MSE: 4.1144643, MAE: 1.3599629
Epoch 223: MSE: 4.909648, MAE: 1.462206
Epoch 224: MSE: 4.1703167, MAE: 1.3853177
Epoch 225: MSE: 5.649729, MAE: 1.4806412


Epoch 401: MSE: 2.610495, MAE: 1.1128999
Epoch 402: MSE: 2.8085387, MAE: 1.1667833
Epoch 403: MSE: 3.5009725, MAE: 1.4362079
Epoch 404: MSE: 2.7984424, MAE: 1.1148496
Epoch 405: MSE: 2.6698468, MAE: 1.3681138
Epoch 406: MSE: 3.67716, MAE: 1.2124313
Epoch 407: MSE: 3.0693297, MAE: 1.2827505
Epoch 408: MSE: 3.046108, MAE: 1.2541758
Epoch 409: MSE: 3.087418, MAE: 1.2654309
Epoch 410: MSE: 2.515476, MAE: 1.3148875
Epoch 411: MSE: 2.976318, MAE: 1.2386342
Epoch 412: MSE: 2.9019837, MAE: 1.3012873
Epoch 413: MSE: 2.7301676, MAE: 1.2721019
Epoch 414: MSE: 3.8036978, MAE: 1.2635882
Epoch 415: MSE: 2.3211625, MAE: 1.1131544
Epoch 416: MSE: 2.4859173, MAE: 1.2634228
Epoch 417: MSE: 2.96648, MAE: 1.3307147
Epoch 418: MSE: 2.7112005, MAE: 1.33182
Epoch 419: MSE: 3.9461405, MAE: 1.1985836
Epoch 420: MSE: 2.5999777, MAE: 1.1493604
Epoch 421: MSE: 2.1765685, MAE: 1.1486284
Epoch 422: MSE: 3.0443406, MAE: 1.3160597
Epoch 423: MSE: 3.22652, MAE: 1.4282454
Epoch 424: MSE: 3.2868147, MAE: 1.2721475
Epoch

# Test

In [14]:
Context.local.learningPhase = .inference

let multiInputTest = MultiInputs(numerical: [XNumericalTest],
                                 categorical: [XCategoricalTest[0],
                                               XCategoricalTest[1]])

let prediction = model(multiInputTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 0.33494344, MAE: 0.040072046


# Export

In [15]:
// print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
// print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

In [16]:
// let coremlModel = Model(version: 4,
//                         shortDescription: "Regression",
//                         author: "Jacopo Mangiavacchi",
//                         license: "MIT",
//                         userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
//     Input(name: "input", shape: [13])
//     Output(name: "output", shape: [1])
//     NeuralNetwork {
//         InnerProduct(name: "dense1",
//                      input: ["input"],
//                      output: ["outDense1"],
//                      weight: model.layer1.weight.transposed().flattened().scalars,
//                      bias: model.layer1.bias.flattened().scalars,
//                      inputChannels: 13,
//                      outputChannels: 64)
//         ReLu(name: "Relu1",
//              input: ["outDense1"],
//              output: ["outRelu1"])
//         InnerProduct(name: "dense2",
//                      input: ["outRelu1"],
//                      output: ["outDense2"],
//                      weight: model.layer2.weight.transposed().flattened().scalars,
//                      bias: model.layer2.bias.flattened().scalars,
//                      inputChannels: 64,
//                      outputChannels: 32)
//         ReLu(name: "Relu2",
//              input: ["outDense2"],
//              output: ["outRelu2"])
//         InnerProduct(name: "dense3",
//                      input: ["outRelu2"],
//                      output: ["output"],
//                      weight: model.layer3.weight.transposed().flattened().scalars,
//                      bias: model.layer3.bias.flattened().scalars,
//                      inputChannels: 32,
//                      outputChannels: 1)
//     }
// }

In [17]:
// let coreMLData = coremlModel.coreMLData
// try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))